<a href="https://colab.research.google.com/github/econ105/AI/blob/main/Python_REITsconstructiondeveloperconstrictionmaterials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings

# Suppress yfinance warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# COMPLETE list with FULL NAMES - Verified active HKEX listings as of 2026
stock_list = {
    # REITs (Real Estate Investment Trusts)
    '0823.HK': {'name': 'Link Real Estate Investment Trust', 'category': 'REIT'},
    '0778.HK': {'name': 'Fortune Real Estate Investment Trust', 'category': 'REIT'},
    '2778.HK': {'name': 'Champion Real Estate Investment Trust', 'category': 'REIT'},
    '0838.HK': {'name': 'Yuexiu Real Estate Investment Trust', 'category': 'REIT'},
    '1426.HK': {'name': 'Spring Real Estate Investment Trust', 'category': 'REIT'},
    '0808.HK': {'name': 'Prosperity Real Estate Investment Trust', 'category': 'REIT'},
    '0435.HK': {'name': 'Sunlight Real Estate Investment Trust', 'category': 'REIT'},

    # Construction / Engineering
    '2038.HK': {'name': 'China State Construction International Holdings', 'category': 'Construction'},
    '1666.HK': {'name': 'China Communications Construction Company', 'category': 'Construction'},
    '1800.HK': {'name': 'China Communications Services Holdings', 'category': 'Construction'},
    '1186.HK': {'name': 'China CAMC Engineering', 'category': 'Construction'},

    # Construction Materials / Building Materials
    '0914.HK': {'name': 'Anhui Conch Cement Company Limited', 'category': 'Materials'},
    '3323.HK': {'name': 'China National Building Material Company', 'category': 'Materials'},
    '2233.HK': {'name': 'West China Cement Limited', 'category': 'Materials'},
    '2269.HK': {'name': 'Kingboard Holdings Limited', 'category': 'Materials'},

    # Property Developers
    '0016.HK': {'name': 'Sun Hung Kai Properties Limited', 'category': 'Developer'},
    '0012.HK': {'name': 'Henderson Land Development Company Limited', 'category': 'Developer'},
    '0017.HK': {'name': 'New World Development Company Limited', 'category': 'Developer'},
    '0384.HK': {'name': 'China Overseas Land & Investment Limited', 'category': 'Developer'},
    '1109.HK': {'name': 'China Resources Land Limited', 'category': 'Developer'},

    # Real Estate ETFs
    '03142.HK': {'name': 'CSOP iEdge S-REIT Leaders ETF', 'category': 'ETF'},
    '08347.HK': {'name': 'Hang Seng REIT ETF', 'category': 'ETF'},
    '03069.HK': {'name': 'Mirae Asset Hong Kong REIT ETF', 'category': 'ETF'}
}

# Extract tickers
all_tickers = list(stock_list.keys())
print(f"Downloading historical data for {len(all_tickers)} Hong Kong stocks/ETFs/REITs from available date...")

# MAXIMUM historical data - yfinance limitation is ~30+ years, not 1900
end_date = datetime.now()
start_date = datetime(1990, 1, 1)  # Earliest realistic for most HK stocks

# Download with comprehensive error handling
successful_data = {}
failed_tickers = []

for ticker in all_tickers:
    try:
        print(f"Fetching {stock_list[ticker]['name']} ({ticker})...", end=" ")
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date, auto_adjust=True)

        if len(hist) > 50:  # Need minimum data
            successful_data[ticker] = hist['Close']
            print(f"✓ {len(hist)} days")
        else:
            failed_tickers.append(ticker)
            print("✗ insufficient data")

    except Exception as e:
        failed_tickers.append(ticker)
        print("✗ failed")

print(f"\n✓ Successfully loaded {len(successful_data)} tickers")
if failed_tickers:
    print(f"✗ Failed: {', '.join([stock_list[t]['name'] for t in failed_tickers])}")

if not successful_data:
    print("No data available. Exiting.")
    exit()

# Combine data
data = pd.DataFrame(successful_data).dropna(how='all')
data = data.dropna(axis=1, thresh=len(data)*0.7)  # Keep columns with 70% data

print(f"\nData range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")

# Calculate comprehensive metrics
prices_current = data.iloc[-1]
prices_first = data.iloc[0]

# Total Return since inception
total_return = ((prices_current / prices_first) - 1) * 100

# 1Y, 3Y, 5Y returns
def calc_period_return(data, years):
    cutoff = data.index[-1] - timedelta(days=years*365.25)
    period_data = data[data.index >= cutoff]
    if len(period_data) > 20:
        return ((period_data.iloc[-1] / period_data.iloc[0]) - 1) * 100
    return np.nan

returns_1y = pd.Series([calc_period_return(data[[t]], 1)[0] for t in data.columns],
                      index=data.columns)
returns_3y = pd.Series([calc_period_return(data[[t]], 3)[0] for t in data.columns],
                      index=data.columns)
returns_5y = pd.Series([calc_period_return(data[[t]], 5)[0] for t in data.columns],
                      index=data.columns)

# Volatility (annualized, 1Y)
daily_returns = data.pct_change().tail(252)
volatility = daily_returns.std() * np.sqrt(252) * 100

# Sharpe Ratio (assuming 2% risk-free rate)
excess_returns = daily_returns.mean() * 252 - 0.02
sharpe = excess_returns / volatility

# Create detailed comparison table
comparison_data = []
for ticker in data.columns:
    info = stock_list[ticker]
    comparison_data.append({
        'Ticker': ticker,
        'Name': info['name'],
        'Category': info['category'],
        'Price (HKD)': prices_current[ticker],
        'Total Return (%)': total_return[ticker],
        '1Y Return (%)': returns_1y[ticker],
        '3Y Return (%)': returns_3y[ticker],
        '5Y Return (%)': returns_5y[ticker],
        'Volatility (%)': volatility[ticker],
        'Sharpe Ratio': sharpe[ticker]
    })

comparison = pd.DataFrame(comparison_data).round(2)

# Sort by 1Y return
comparison_sorted = comparison.sort_values('1Y Return (%)', ascending=False)

print("\n" + "="*120)
print("COMPLETE HONG KONG REITs, CONSTRUCTION, MATERIALS, DEVELOPERS & RE ETFs ANALYSIS")
print("="*120)
print(comparison_sorted.to_string(index=False))

# Category performance summary
print("\n" + "="*80)
print("CATEGORY PERFORMANCE SUMMARY (1Y Returns)")
print("="*80)
category_summary = comparison.groupby('Category')['1Y Return (%)'].agg(['count', 'mean', 'max']).round(2)
category_summary.columns = ['Count', 'Avg 1Y (%)', 'Best 1Y (%)']
print(category_summary)

# Top performers
print("\n" + "="*50)
print("TOP 5 PERFORMERS (1Y)")
print("="*50)
print(comparison.nlargest(5, '1Y Return (%)')[['Name', 'Category', '1Y Return (%)']].to_string(index=False))

print(f"\n📊 Analysis complete | Data period: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
print(f"💰 Total tickers analyzed: {len(comparison)}")


Fetching Link Real Estate Investment Trust (0823.HK)... ✓ 4496 days
Fetching Fortune Real Estate Investment Trust (0778.HK)... ✓ 3887 days
Fetching Champion Real Estate Investment Trust (2778.HK)... ✓ 4496 days
Fetching Yuexiu Real Estate Investment Trust (0838.HK)... ✓ 5109 days
Fetching Spring Real Estate Investment Trust (1426.HK)... ✓ 2989 days
Fetching Prosperity Real Estate Investment Trust (0808.HK)... ✓ 4496 days
Fetching Sunlight Real Estate Investment Trust (0435.HK)... ✓ 4496 days
Fetching China State Construction International Holdings (2038.HK)... ✓ 5171 days
Fetching China Communications Construction Company (1666.HK)... ✓ 6283 days
Fetching China Communications Services Holdings (1800.HK)... ✓ 4709 days
Fetching China CAMC Engineering (1186.HK)... ✓ 5011 days
Fetching Anhui Conch Cement Company Limited (0914.HK)... ✓ 6498 days
Fetching China National Building Material Company (3323.HK)... ✓ 4892 days
Fetching West China Cement Limited (2233.HK)... ✓ 4718 days
Fetching Ki

ERROR:yfinance:$03142.HK: possibly delisted; no timezone found
ERROR:yfinance:$08347.HK: possibly delisted; no timezone found
ERROR:yfinance:$03069.HK: possibly delisted; no timezone found


✓ 6498 days
Fetching CSOP iEdge S-REIT Leaders ETF (03142.HK)... ✗ insufficient data
Fetching Hang Seng REIT ETF (08347.HK)... ✗ insufficient data
Fetching Mirae Asset Hong Kong REIT ETF (03069.HK)... ✗ insufficient data

✓ Successfully loaded 20 tickers
✗ Failed: CSOP iEdge S-REIT Leaders ETF, Hang Seng REIT ETF, Mirae Asset Hong Kong REIT ETF

Data range: 2000-01-04 to 2026-01-23

COMPLETE HONG KONG REITs, CONSTRUCTION, MATERIALS, DEVELOPERS & RE ETFs ANALYSIS
 Ticker                                            Name     Category  Price (HKD)  Total Return (%)  1Y Return (%)  3Y Return (%)  5Y Return (%)  Volatility (%)  Sharpe Ratio
2038.HK China State Construction International Holdings Construction        20.02               NaN         153.42         135.53          39.03           54.12          0.02
2233.HK                       West China Cement Limited    Materials         3.45               NaN         142.53         306.48         315.74           49.76          0.02
0017.HK 